In [1]:
import os
import sys
import re
import csv

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.wkt
import usaddress
from number_parser import parse_ordinal

In [2]:
os.chdir('..')
abs_path = os.getcwd()

zcta_folder = os.path.join(abs_path, 'data/tl_2023_us_zcta520_clean')
zcta_file = 'tl_2023_us_zcta520_clean.shp'
zcta_path = os.path.join(zcta_folder, zcta_file)

OMOP_county_full_path = os.path.join(abs_path, 'output', 'OMOP_county_full.csv')

zcta_crosswalk_file = "tab20_zcta520_county20_natl.txt"
zcta_crosswalk_path = os.path.join(abs_path, "data", zcta_crosswalk_file)

In [3]:
def create_dir(save_dir):
    """
    Creates directory if it does not exist
         
    Parameters
    ----------
        save_dir (str): path of desired output directory
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

# ZCTA County Match
* ZCTA 2023
* ZCTA 2020 Crosswalk

In [4]:
omop_county_full = pd.read_csv(OMOP_county_full_path)
zcta_crosswalk = pd.read_csv(zcta_crosswalk_path, sep="|")

C:\Users\bchan\AppData\Local\Temp\ipykernel_24952\1393104656.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_county_full = pd.read_csv(OMOP_county_full_path)


In [31]:
# drop unnecessary columns
zcta_crosswalk = zcta_crosswalk.drop(columns=["OID_ZCTA5_20", "AREALAND_ZCTA5_20", "AREAWATER_ZCTA5_20", "MTFCC_ZCTA5_20", "CLASSFP_ZCTA5_20", "FUNCSTAT_ZCTA5_20", 
                                              "AREALAND_COUNTY_20", "AREAWATER_COUNTY_20", "MTFCC_COUNTY_20", "CLASSFP_COUNTY_20", "FUNCSTAT_COUNTY_20", "AREALAND_PART", "AREAWATER_PART"])
zcta_crosswalk

,GEOID_ZCTA5_20,NAMELSAD_ZCTA5_20,OID_COUNTY_20,GEOID_COUNTY_20,NAMELSAD_COUNTY_20
0,NaN,NaN,27590114112812,1003,Baldwin County
1,NaN,NaN,2759099719300,1007,Bibb County
2,NaN,NaN,27590103020886,1015,Calhoun County
3,NaN,NaN,27590336389978,1021,Chilton County
4,NaN,NaN,2759075862059,1025,Clarke County
...,...,...,...,...,...
47858,99923.0,ZCTA5 99923,275903025880880,2198,Prince of Wales-Hyder Census Area
47859,99925.0,ZCTA5 99925,275903025880880,2198,Prince of Wales-Hyder Census Area
47860,99926.0,ZCTA5 99926,275903025880880,2198,Prince of Wales-Hyder Census Area
47861,99927.0,ZCTA5 99927,275903025880880,2198,Prince of Wales-Hyder Census Area


In [32]:
zcta_shapes = gpd.read_file(zcta_path)
zcta_shapes = zcta_shapes.to_crs("EPSG:4326")
zcta_shapes

,ZCTA5CE20,GEOID20,GEOIDFQ20,INTPTLAT20,INTPTLON20,geometry
0,47236,47236,860Z200US47236,+39.1517426,-085.7252769,"POLYGON ((-85.73410 39.15597, -85.72794 39.156..."
1,47870,47870,860Z200US47870,+39.3701518,-087.4735141,"POLYGON ((-87.47415 39.37016, -87.47410 39.370..."
2,47851,47851,860Z200US47851,+39.5735839,-087.2459559,"POLYGON ((-87.24769 39.57450, -87.24711 39.574..."
3,47337,47337,860Z200US47337,+39.8027537,-085.4372850,"POLYGON ((-85.44356 39.80328, -85.44345 39.803..."
4,47435,47435,860Z200US47435,+39.2657557,-086.2951577,"POLYGON ((-86.29592 39.26547, -86.29592 39.266..."
...,...,...,...,...,...,...
33786,37932,37932,860Z200US37932,+35.9172993,-084.1987873,"POLYGON ((-84.27347 35.93928, -84.27287 35.940..."
33787,37341,37341,860Z200US37341,+35.2199309,-085.0730025,"POLYGON ((-85.15090 35.11231, -85.15088 35.112..."
33788,37849,37849,860Z200US37849,+36.0540502,-084.0484876,"POLYGON ((-84.14857 36.04234, -84.14240 36.046..."
33789,37754,37754,860Z200US37754,+36.1390993,-084.0298007,"POLYGON ((-84.10549 36.11168, -84.10543 36.111..."


In [ ]:
# save ZCTA shapefile after dropping columns
# ONLY RUN ONCE
zcta_shapes = zcta_shapes.drop(columns=["CLASSFP20", "MTFCC20", "FUNCSTAT20", "ALAND20", "AWATER20"])

# shp_file = dict[fname].set_geometry('source_centroid')
save_dir = os.path.join(abs_path, 'data')        
save_path = os.path.join(save_dir, "tl_2023_us_zcta520_clean")
create_dir(save_path)

zcta_shapes.to_file(save_path, driver='ESRI Shapefile')

In [33]:
zcta_crosswalk['ZCTA5CE20'] = zcta_crosswalk.GEOID_ZCTA5_20.apply(lambda x: str(x).rstrip(".0") if ".0" in str(x) else str(x))
# convert to string without trailing zeros and decimal

zcta_crosswalk

,GEOID_ZCTA5_20,NAMELSAD_ZCTA5_20,OID_COUNTY_20,GEOID_COUNTY_20,NAMELSAD_COUNTY_20,ZCTA5CE20
0,NaN,NaN,27590114112812,1003,Baldwin County,nan
1,NaN,NaN,2759099719300,1007,Bibb County,nan
2,NaN,NaN,27590103020886,1015,Calhoun County,nan
3,NaN,NaN,27590336389978,1021,Chilton County,nan
4,NaN,NaN,2759075862059,1025,Clarke County,nan
...,...,...,...,...,...,...
47858,99923.0,ZCTA5 99923,275903025880880,2198,Prince of Wales-Hyder Census Area,99923
47859,99925.0,ZCTA5 99925,275903025880880,2198,Prince of Wales-Hyder Census Area,99925
47860,99926.0,ZCTA5 99926,275903025880880,2198,Prince of Wales-Hyder Census Area,99926
47861,99927.0,ZCTA5 99927,275903025880880,2198,Prince of Wales-Hyder Census Area,99927


In [34]:
zcta_county_name = zcta_shapes.merge(zcta_crosswalk, how="left", on="ZCTA5CE20")
zcta_county_name['zip'] = zcta_county_name.ZCTA5CE20
zcta_county_name

,ZCTA5CE20,GEOID20,GEOIDFQ20,INTPTLAT20,INTPTLON20,geometry,GEOID_ZCTA5_20,NAMELSAD_ZCTA5_20,OID_COUNTY_20,GEOID_COUNTY_20,NAMELSAD_COUNTY_20,zip
0,47236,47236,860Z200US47236,+39.1517426,-085.7252769,"POLYGON ((-85.73410 39.15597, -85.72794 39.156...",47236.0,ZCTA5 47236,2.759010e+13,18005.0,Bartholomew County,47236
1,47870,47870,860Z200US47870,+39.3701518,-087.4735141,"POLYGON ((-87.47415 39.37016, -87.47410 39.370...",NaN,NaN,NaN,NaN,NaN,47870
2,47851,47851,860Z200US47851,+39.5735839,-087.2459559,"POLYGON ((-87.24769 39.57450, -87.24711 39.574...",47851.0,ZCTA5 47851,2.759035e+13,18167.0,Vigo County,47851
3,47337,47337,860Z200US47337,+39.8027537,-085.4372850,"POLYGON ((-85.44356 39.80328, -85.44345 39.803...",47337.0,ZCTA5 47337,2.759011e+13,18065.0,Henry County,47337
4,47435,47435,860Z200US47435,+39.2657557,-086.2951577,"POLYGON ((-86.29592 39.26547, -86.29592 39.266...",47435.0,ZCTA5 47435,2.759010e+13,18013.0,Brown County,47435
...,...,...,...,...,...,...,...,...,...,...,...,...
45178,37754,37754,860Z200US37754,+36.1390993,-084.0298007,"POLYGON ((-84.10549 36.11168, -84.10543 36.111...",37754.0,ZCTA5 37754,2.759026e+13,47001.0,Anderson County,37754
45179,37754,37754,860Z200US37754,+36.1390993,-084.0298007,"POLYGON ((-84.10549 36.11168, -84.10543 36.111...",37754.0,ZCTA5 37754,2.759023e+13,47093.0,Knox County,37754
45180,37754,37754,860Z200US37754,+36.1390993,-084.0298007,"POLYGON ((-84.10549 36.11168, -84.10543 36.111...",37754.0,ZCTA5 37754,2.759054e+13,47173.0,Union County,37754
45181,37806,37806,860Z200US37806,+36.0846931,-083.7279865,"MULTIPOLYGON (((-83.78542 36.08103, -83.78461 ...",37806.0,ZCTA5 37806,2.759022e+13,47057.0,Grainger County,37806


In [35]:
# add point geometry column that matches expected gpd format
omop_county_full['geometry'] = omop_county_full.apply(lambda x: "POINT (" + str(x.longitude) + " " + str(x.latitude) + ")", axis=1)

# convert to geopandas DataFrame and set `geometry` centroid 
omop_county_gdf = gpd.GeoDataFrame(omop_county_full, geometry=omop_county_full['geometry'].apply(shapely.wkt.loads))
omop_county_gdf.crs = "EPSG:4326"
omop_county_gdf

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry
0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.04352 42.33547)
1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.16494 42.70621)
2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS,POINT (-74.16282 40.76993)
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74782 42.79853)
4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS,POINT (-70.11378 44.42861)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78795,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.09517 34.04173)
78796,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.10132 33.96680)
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90222 34.06883)
78798,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS,POINT (-84.49411 37.99651)


In [36]:
# spatial join omop county file to zcta
omop_county_zcta = omop_county_gdf.sjoin(zcta_county_name, how='left')
omop_county_zcta["county"] =  omop_county_zcta.county.apply(lambda x: str(x) + " County")
omop_county_zcta

,Location_id,address_1,address_2,city,state,zip_left,county,location_source_value,latitude,longitude,...,GEOID20,GEOIDFQ20,INTPTLAT20,INTPTLON20,GEOID_ZCTA5_20,NAMELSAD_ZCTA5_20,OID_COUNTY_20,GEOID_COUNTY_20,NAMELSAD_COUNTY_20,zip_right
0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk County,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,...,02127,860Z200US02127,+42.3353007,-071.0382163,NaN,NaN,NaN,NaN,NaN,02127
1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex County,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,...,01840,860Z200US01840,+42.7067633,-071.1604026,NaN,NaN,NaN,NaN,NaN,01840
2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex County,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,...,07104,860Z200US07104,+40.7677132,-074.1683498,NaN,NaN,NaN,NaN,NaN,07104
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,...,14127,860Z200US14127,+42.7528037,-078.7396939,14127.0,ZCTA5 14127,2.759012e+13,36029.0,Erie County,14127
4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin County,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,...,04254,860Z200US04254,+44.4453367,-070.1380761,NaN,NaN,NaN,NaN,NaN,04254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78796,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,...,30096,860Z200US30096,+33.9743018,-084.1453842,30096.0,ZCTA5 30096,2.759044e+13,13135.0,Gwinnett County,30096
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,...,30019,860Z200US30019,+33.9756343,-083.8837695,30019.0,ZCTA5 30019,2.759044e+13,13135.0,Gwinnett County,30019
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,...,30019,860Z200US30019,+33.9756343,-083.8837695,30019.0,ZCTA5 30019,2.759042e+13,13297.0,Walton County,30019
78798,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,...,40502,860Z200US40502,+38.0109529,-084.4831908,40502.0,ZCTA5 40502,2.759010e+13,21067.0,Fayette County,40502


In [37]:
omop_county_zcta["county_match"] = omop_county_zcta.apply(lambda x: 1 if x.county == x.NAMELSAD_COUNTY_20 else 0, axis=1)
omop_county_zcta

,Location_id,address_1,address_2,city,state,zip_left,county,location_source_value,latitude,longitude,...,GEOIDFQ20,INTPTLAT20,INTPTLON20,GEOID_ZCTA5_20,NAMELSAD_ZCTA5_20,OID_COUNTY_20,GEOID_COUNTY_20,NAMELSAD_COUNTY_20,zip_right,county_match
0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk County,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,...,860Z200US02127,+42.3353007,-071.0382163,NaN,NaN,NaN,NaN,NaN,02127,0
1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex County,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,...,860Z200US01840,+42.7067633,-071.1604026,NaN,NaN,NaN,NaN,NaN,01840,0
2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex County,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,...,860Z200US07104,+40.7677132,-074.1683498,NaN,NaN,NaN,NaN,NaN,07104,0
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,...,860Z200US14127,+42.7528037,-078.7396939,14127.0,ZCTA5 14127,2.759012e+13,36029.0,Erie County,14127,1
4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin County,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,...,860Z200US04254,+44.4453367,-070.1380761,NaN,NaN,NaN,NaN,NaN,04254,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78796,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,...,860Z200US30096,+33.9743018,-084.1453842,30096.0,ZCTA5 30096,2.759044e+13,13135.0,Gwinnett County,30096,1
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,...,860Z200US30019,+33.9756343,-083.8837695,30019.0,ZCTA5 30019,2.759044e+13,13135.0,Gwinnett County,30019,1
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,...,860Z200US30019,+33.9756343,-083.8837695,30019.0,ZCTA5 30019,2.759042e+13,13297.0,Walton County,30019,0
78798,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,...,860Z200US40502,+38.0109529,-084.4831908,40502.0,ZCTA5 40502,2.759010e+13,21067.0,Fayette County,40502,1


In [38]:
omop_county_zcta.columns

Index(['Location_id', 'address_1', 'address_2', 'city', 'state', 'zip_left',
       'county', 'location_source_value', 'latitude', 'longitude',
       'address_type', 'state_abbr', 'flag', 'geometry', 'index_right',
       'ZCTA5CE20', 'GEOID20', 'GEOIDFQ20', 'INTPTLAT20', 'INTPTLON20',
       'GEOID_ZCTA5_20', 'NAMELSAD_ZCTA5_20', 'OID_COUNTY_20',
       'GEOID_COUNTY_20', 'NAMELSAD_COUNTY_20', 'zip_right', 'county_match'],
      dtype='object')

In [40]:
omop_county_zcta = omop_county_zcta.drop(columns = ["GEOIDFQ20", "INTPTLAT20", "INTPTLON20", "NAMELSAD_ZCTA5_20", "OID_COUNTY_20", "zip_right" ,"GEOID_ZCTA5_20", "GEOID_COUNTY_20", "index_right"])
omop_county_zcta = omop_county_zcta.rename({"zip_left": "zip"})
omop_county_zcta

,Location_id,address_1,address_2,city,state,zip_left,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry,ZCTA5CE20,GEOID20,NAMELSAD_COUNTY_20,county_match
0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk County,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.04352 42.33547),02127,02127,NaN,0
1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex County,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.16494 42.70621),01840,01840,NaN,0
2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex County,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS,POINT (-74.16282 40.76993),07104,07104,NaN,0
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74782 42.79853),14127,14127,Erie County,1
4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin County,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS,POINT (-70.11378 44.42861),04254,04254,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78796,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.10132 33.96680),30096,30096,Gwinnett County,1
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90222 34.06883),30019,30019,Gwinnett County,1
78797,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90222 34.06883),30019,30019,Walton County,0
78798,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS,POINT (-84.49411 37.99651),40502,40502,Fayette County,1


In [41]:
omop_county_zcta.county_match.value_counts()

county_match
1    65697
0    36157
Name: count, dtype: int64

In [42]:
omop_county_zcta_path = os.path.join(abs_path, 'output', 'omop_county_zcta.csv')
omop_county_zcta.to_csv(omop_county_zcta_path, index=False)

In [37]:
omop_county_zcta_path = os.path.join(abs_path, 'output', 'omop_county_zcta.csv')
omop_county_zcta = pd.read_csv(omop_county_zcta_path)

C:\Users\bchan\AppData\Local\Temp\ipykernel_24952\2403018500.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_county_zcta = pd.read_csv(omop_county_zcta_path)


In [8]:
omop_county_zcta.state.nunique()

57

In [9]:
omop_county_zcta_match = omop_county_zcta.loc[omop_county_zcta.county_match == 1]
# omop_county_zcta_match = omop_county_zcta_match.drop(columns = ["GEOIDFQ20", "INTPTLAT20", "INTPTLON20", "NAMELSAD_ZCTA5_20", "OID_COUNTY_20", "zip_right" ,"Unnamed: 0", "GEOID_ZCTA5_20", "GEOID_COUNTY_20"])
omop_county_zcta_match = omop_county_zcta_match.rename({"zip_left": "zip"})
print(omop_county_zcta_match.columns)
omop_county_zcta_match

Index(['Location_id', 'address_1', 'address_2', 'city', 'state', 'zip_left',
       'county', 'location_source_value', 'latitude', 'longitude',
       'address_type', 'state_abbr', 'flag', 'geometry', 'ZCTA5CE20',
       'GEOID20', 'NAMELSAD_COUNTY_20', 'county_match'],
      dtype='object')


,Location_id,address_1,address_2,city,state,zip_left,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry,ZCTA5CE20,GEOID20,NAMELSAD_COUNTY_20,county_match
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74781599997843 42.79853499984358),14127.0,14127.0,Erie County,1
192,263,562 Wingate Dr,NaN,East Meadow,New York,11554,Nassau County,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.52838899973285 40.71011299966788),11554.0,11554.0,Nassau County,1
195,279,8 Zoar Ave,NaN,Albany,New York,12209,Albany County,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.78899100027446 42.63906699972181),12209.0,12209.0,Albany County,1
200,300,15402 41St Ave,NaN,Flushing,New York,11354,Queens County,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.81094400044168 40.76300300008896),11354.0,11354.0,Queens County,1
201,305,5110 19Th Ave,NaN,Brooklyn,New York,11204,Kings County,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.98001299964284 40.62562299970931),11204.0,11204.0,Kings County,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101848,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett County,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.09517421962246 34.04172694900986),30024.0,30024.0,Gwinnett County,1
101849,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.10131848955332 33.96679709299809),30096.0,30096.0,Gwinnett County,1
101850,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90221543184028 34.06883234729177),30019.0,30019.0,Gwinnett County,1
101852,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS,POINT (-84.49410603994914 37.99650802456039),40502.0,40502.0,Fayette County,1


In [10]:
# capitalize only first letter
omop_county_zcta_match['address_1'] = omop_county_zcta_match.address_1.apply(lambda x: str(x).strip().title())
omop_county_zcta_match['address_2'] = omop_county_zcta_match.address_2.apply(lambda x: str(x).strip().title() if not np.NaN else x)
omop_county_zcta_match['city'] = omop_county_zcta_match.city.apply(lambda x: str(x).strip().title())
# omop_county_zcta_match['location_source_value'] = omop_county_zcta_match.location_source_value.apply(lambda x: str(x).strip().title())
omop_county_zcta_match

,Location_id,address_1,address_2,city,state,zip_left,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry,ZCTA5CE20,GEOID20,NAMELSAD_COUNTY_20,county_match
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74781599997843 42.79853499984358),14127.0,14127.0,Erie County,1
192,263,562 Wingate Dr,NaN,East Meadow,New York,11554,Nassau County,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.52838899973285 40.71011299966788),11554.0,11554.0,Nassau County,1
195,279,8 Zoar Ave,NaN,Albany,New York,12209,Albany County,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.78899100027446 42.63906699972181),12209.0,12209.0,Albany County,1
200,300,15402 41St Ave,NaN,Flushing,New York,11354,Queens County,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.81094400044168 40.76300300008896),11354.0,11354.0,Queens County,1
201,305,5110 19Th Ave,NaN,Brooklyn,New York,11204,Kings County,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.98001299964284 40.62562299970931),11204.0,11204.0,Kings County,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101848,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett County,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.09517421962246 34.04172694900986),30024.0,30024.0,Gwinnett County,1
101849,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.10131848955332 33.96679709299809),30096.0,30096.0,Gwinnett County,1
101850,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90221543184028 34.06883234729177),30019.0,30019.0,Gwinnett County,1
101852,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS,POINT (-84.49410603994914 37.99650802456039),40502.0,40502.0,Fayette County,1


In [41]:
OMOP_county_zcta_path = os.path.join(abs_path, 'output', 'OMOP_county_zcta.csv')
omop_county_zcta.to_csv(OMOP_county_zcta_path, index=False)

In [60]:
OMOP_county_zcta_match_path = os.path.join(abs_path, 'output', 'OMOP_county_zcta_match.csv')
omop_county_zcta_match.to_csv(OMOP_county_zcta_match_path, index=False)

In [44]:
# merge on zip
omop_county_zcta_zip = omop_county_gdf.merge(zcta_county_name, how='left', on='zip')
omop_county_zcta_zip["county"] =  omop_county_zcta_zip.county.apply(lambda x: str(x) + " County")
omop_county_zcta_zip["county_match"] = omop_county_zcta_zip.apply(lambda x: 1 if x.county == x.NAMELSAD_COUNTY_20 else 0, axis=1)

# omop_county_zcta_zip = omop_county_zcta_zip.loc[omop_county_zcta_zip.county_match == 1]
omop_county_zcta_zip

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,...,GEOIDFQ20,INTPTLAT20,INTPTLON20,geometry_y,GEOID_ZCTA5_20,NAMELSAD_ZCTA5_20,OID_COUNTY_20,GEOID_COUNTY_20,NAMELSAD_COUNTY_20,county_match
0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk County,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,...,860Z200US02127,+42.3353007,-071.0382163,"POLYGON ((-71.06355 42.33079, -71.06291 42.331...",NaN,NaN,NaN,NaN,NaN,0
1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex County,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,...,860Z200US01840,+42.7067633,-071.1604026,"POLYGON ((-71.17248 42.70787, -71.17261 42.707...",NaN,NaN,NaN,NaN,NaN,0
2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex County,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,...,860Z200US07104,+40.7677132,-074.1683498,"POLYGON ((-74.18521 40.75686, -74.18516 40.757...",NaN,NaN,NaN,NaN,NaN,0
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,...,860Z200US14127,+42.7528037,-078.7396939,"POLYGON ((-78.80954 42.77168, -78.80828 42.772...",14127.0,ZCTA5 14127,2.759012e+13,36029.0,Erie County,1
4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin County,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,...,860Z200US04254,+44.4453367,-070.1380761,"POLYGON ((-70.19976 44.47826, -70.19885 44.478...",NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96107,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett County,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0
96108,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0
96109,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0
96110,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,0


In [46]:
# omop_county_zcta_zip_match = omop_county_zcta_zip.loc[omop_county_zcta_zip.county_match == 1]
omop_county_zcta_zip = omop_county_zcta_zip.drop(columns = ["GEOIDFQ20", "INTPTLAT20", "INTPTLON20", "NAMELSAD_ZCTA5_20", "OID_COUNTY_20", 
                                                            "GEOID_ZCTA5_20", "GEOID_COUNTY_20", "geometry_y", "GEOID20"])
omop_county_zcta_zip = omop_county_zcta_zip.rename({"geometry_x": "geometry"})
omop_county_zcta_zip

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry_x,ZCTA5CE20,NAMELSAD_COUNTY_20,county_match
0,1,523 E Broadway,NaN,South Boston,Massachusetts,02127,Suffolk County,"523 E BROADWAY, SOUTH BOSTON, MA 02127",42.335472,-71.043522,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.04352 42.33547),02127,NaN,0
1,2,454 Essex St,NaN,Lawrence,Massachusetts,01840,Essex County,"454 ESSEX ST, LAWRENCE, MA 01840",42.706213,-71.164940,Street Address,MA,SUCCESSFUL ADDRESS,POINT (-71.16494 42.70621),01840,NaN,0
2,3,569 Broadway,NaN,Newark,New Jersey,07104,Essex County,"569 BROADWAY, NEWARK, NJ 07104",40.769935,-74.162821,Street Address,NJ,SUCCESSFUL ADDRESS,POINT (-74.16282 40.76993),07104,NaN,0
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74782 42.79853),14127,Erie County,1
4,5,431 Campground Rd,NaN,Livermore Fls,Maine,04254,Androscoggin County,"431 CAMPGROUND RD, LIVERMORE FLS, ME 04254",44.428610,-70.113777,Street Address,ME,SUCCESSFUL ADDRESS,POINT (-70.11378 44.42861),04254,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96107,104098,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett County,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.09517 34.04173),NaN,NaN,0
96108,104099,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.10132 33.96680),NaN,NaN,0
96109,104100,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90222 34.06883),NaN,NaN,0
96110,104101,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS,POINT (-84.49411 37.99651),NaN,NaN,0


In [47]:
omop_county_zcta_zip_path = os.path.join(abs_path, 'output', 'omop_county_zcta_zip.csv')
omop_county_zcta_zip.to_csv(omop_county_zcta_zip_path, index=False)

In [45]:
omop_county_zcta_zip_match.columns

Index(['location_id', 'address_1', 'address_2', 'city', 'state', 'zip',
       'county', 'location_source_value', 'latitude', 'longitude',
       'address_type', 'state_abbr', 'flag', 'geometry', 'ZCTA5CE20',
       'NAMELSAD_COUNTY_20', 'county_match'],
      dtype='object')

In [99]:
omop_county_zcta_zip_match = omop_county_zcta_zip.loc[omop_county_zcta_zip.county_match == 1]
omop_county_zcta_zip_match = omop_county_zcta_zip_match.drop(columns = ["GEOIDFQ20", "INTPTLAT20", "INTPTLON20", "NAMELSAD_ZCTA5_20", "OID_COUNTY_20", 
                                                                        "Unnamed: 0", "GEOID_ZCTA5_20", "GEOID_COUNTY_20", "geometry_y", "GEOID20"])
omop_county_zcta_zip_match = omop_county_zcta_zip_match.rename({"geometry_x": "geometry"})
omop_county_zcta_zip_match

,Location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry_x,ZCTA5CE20,NAMELSAD_COUNTY_20,county_match
3,4,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74782 42.79853),14127,Erie County,1
192,263,562 Wingate Dr,NaN,East Meadow,New York,11554,Nassau County,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.52839 40.71011),11554,Nassau County,1
195,279,8 Zoar Ave,NaN,Albany,New York,12209,Albany County,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.78899 42.63907),12209,Albany County,1
200,300,15402 41St Ave,NaN,Flushing,New York,11354,Queens County,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.81094 40.76300),11354,Queens County,1
201,305,5110 19Th Ave,NaN,Brooklyn,New York,11204,Kings County,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.98001 40.62562),11204,Kings County,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82843,85594,5000 N Bowes Rd,NaN,Tucson,Arizona,85749,Pima County,"5000 N BOWES RD, TUCSON, AZ 85749",32.298797,-110.788349,Street Address,AZ,SUCCESSFUL ADDRESS,POINT (-110.78835 32.29880),85749,Pima County,1
82844,85596,1303 Paseo Del Canon East,NaN,Taos,New Mexico,87571,Taos County,"1303 PASEO DEL CANON EAST, TAOS, NM 87571",36.375292,-105.568573,Street Address,NM,SUCCESSFUL ADDRESS,POINT (-105.56857 36.37529),87571,Taos County,1
82845,85599,12606 E Main St,NaN,Mayer,Arizona,86333,Yavapai County,"12606 E MAIN ST, MAYER, AZ 86333",34.346322,-112.157189,Street Address,AZ,SUCCESSFUL ADDRESS,POINT (-112.15719 34.34632),86333,Yavapai County,1
82846,85600,4300 Blake Rd Sw,NaN,Albuquerque,New Mexico,87121,Bernalillo County,"4300 BLAKE RD SW, ALBUQUERQUE, NM 87121",35.037163,-106.723264,Street Address,NM,SUCCESSFUL ADDRESS,POINT (-106.72326 35.03716),87121,Bernalillo County,1


In [100]:
OMOP_county_zcta_zip_match_path = os.path.join(abs_path, 'output', 'OMOP_county_zcta_zip_match.csv')
omop_county_zcta_zip_match.to_csv(OMOP_county_zcta_zip_match_path, index=False)

# Nominatim Parsing
1. Parse base Nominatim components with `usaddress`
2. Keep only the components needed for Nominatim
3. Merge the Nominatim components to the Publication 28 (OMOP) components
4. Convert all abbreviations for cardinal directions and street type to full strings
5. Join Nominatim components to a full address string

In [13]:
from requests.structures import CaseInsensitiveDict

# for converting cardinal direction abbreviations to full string
cardinal_directions_to_full = CaseInsensitiveDict({"N": "North", "E": "East", "S": "South", "W": "West", 
                                                  "NE": "Northeast", "SE": "Southeast", "NW": "Northwest", "SW": "Southwest"})

# for converting street type abbreviations to full string
street_suffix_to_full = CaseInsensitiveDict({
    'aly': 'Alley',
    'ave': 'Avenue',
    'blvd': 'Boulevard',
    'cir': 'Circle',
    'ct': 'Court',
    'dr': 'Drive',
    'expy': 'Expressway', 
    'grv': 'Grove', 
    'grve': 'Grove',
    'hwy': 'Highway',
    'ln': 'Lane',
    'pkwy': 'Parkway',
    'pl': 'Place', 
    'plz': 'Place',
    'rd': 'Road', 
    'spgs': 'Springs',
    'sq': 'Square',
    'st': 'Street',
    'ter': 'Terrace',
    'trl': 'Trail',
    'way': 'Way'
})

In [14]:
# function for case insensitive dictionary replacement using Regex
def replace_words(text, word_dict):
    for key, value in word_dict.items():
        # Use case-insensitive regex for replacement
        text = re.sub(r'\b'+key+'\b', value, text, flags=re.IGNORECASE, count=1)
    return text

def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key], 1)
    return text

def cardinal_direction_lambda(x, word_dict):
    """
    Pass cardinal_directions_to_full
    """
    if str(x) == "nan":
        return np.NaN
    elif x in word_dict.keys():
        return word_dict.get(str(x))
    else:
        return x

def street_suffix_lambda(x, word_dict):
    """
    Pass street_suffix_to_full
    """
    if str(x) == "nan" or str(x) == "Nan":
        return np.NaN
    # elif len(str(x)) > 2:
    #     return str(x)
    elif x in word_dict.keys():
        return word_dict.get(str(x))
    else:
        return x

# def replace_ordinal_numbers(text):
#     """
#     Replace ordinal numbers with full string
#     """
#     re_results = re.findall('(\d+(st|nd|rd|th))', text)
#     for enitre_result, suffix in re_results:
#         num = int(enitre_result[:-len(suffix)])
#         text = text.replace(enitre_result, num2words(num, ordinal=True))
#     return text

def word_to_ordinal(x):
    n = parse_ordinal(x)
    return n

def make_ordinal(x):
    '''
    Convert an integer into its ordinal representation::

        make_ordinal(0)   => '0th'
        make_ordinal(3)   => '3rd'
        make_ordinal(122) => '122nd'
        make_ordinal(213) => '213th'
    '''
    n = word_to_ordinal(x)
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    return str(n) + suffix

In [6]:
# import omop county file that has both zcta and spatial join match by county name
OMOP_county_zcta_zip_match_path = os.path.join(abs_path, 'output', 'OMOP_county_zcta_zip_match.csv')
omop_county_zcta_zip_match = pd.read_csv(OMOP_county_zcta_zip_match_path)
omop_county_zcta_zip_match = omop_county_zcta_zip_match.rename(columns={"geometry_x": "geometry", "Location_id": "location_id"})
omop_county_zcta_zip_match['location_id'] = omop_county_zcta_zip_match.index+1

In [7]:
omop_county_zcta_zip_match

,location_id,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry,ZCTA5CE20,NAMELSAD_COUNTY_20,county_match
0,1,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74781599997843 42.79853499984358),14127,Erie County,1
1,2,562 Wingate Dr,NaN,East Meadow,New York,11554,Nassau County,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.52838899973285 40.71011299966788),11554,Nassau County,1
2,3,8 Zoar Ave,NaN,Albany,New York,12209,Albany County,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.78899100027446 42.63906699972181),12209,Albany County,1
3,4,15402 41St Ave,NaN,Flushing,New York,11354,Queens County,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.81094400044168 40.76300300008896),11354,Queens County,1
4,5,5110 19Th Ave,NaN,Brooklyn,New York,11204,Kings County,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.98001299964284 40.62562299970931),11204,Kings County,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40412,40413,5000 N Bowes Rd,NaN,Tucson,Arizona,85749,Pima County,"5000 N BOWES RD, TUCSON, AZ 85749",32.298797,-110.788349,Street Address,AZ,SUCCESSFUL ADDRESS,POINT (-110.7883488373378 32.29879689494836),85749,Pima County,1
40413,40414,1303 Paseo Del Canon East,NaN,Taos,New Mexico,87571,Taos County,"1303 PASEO DEL CANON EAST, TAOS, NM 87571",36.375292,-105.568573,Street Address,NM,SUCCESSFUL ADDRESS,POINT (-105.568573404848 36.37529177977968),87571,Taos County,1
40414,40415,12606 E Main St,NaN,Mayer,Arizona,86333,Yavapai County,"12606 E MAIN ST, MAYER, AZ 86333",34.346322,-112.157189,Street Address,AZ,SUCCESSFUL ADDRESS,POINT (-112.1571889686942 34.34632218671697),86333,Yavapai County,1
40415,40416,4300 Blake Rd Sw,NaN,Albuquerque,New Mexico,87121,Bernalillo County,"4300 BLAKE RD SW, ALBUQUERQUE, NM 87121",35.037163,-106.723264,Street Address,NM,SUCCESSFUL ADDRESS,POINT (-106.7232635088362 35.03716265606487),87121,Bernalillo County,1


In [8]:
%%time

# run `usaddress` parsing with base component names and append to list
repo = []
for ind, each in omop_county_zcta_zip_match.iterrows():
    try:
        obj = usaddress.tag(each.location_source_value)
        tmp = pd.DataFrame(obj[0], columns=obj[0].keys(), index=[ind])
        tmp['Address_type'] = obj[1]
        tmp['county'] = each.county
        tmp['location_source_value'] = each.location_source_value
        tmp['location_id'] = each.location_id

        # development
        # OMOP_location.loc[ind, 'AddressNumber'] = [tmp['AddressNumber'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePreDirectional'] = [tmp['StreetNamePreDirectional'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePreType'] = [tmp['StreetNamePreType'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetName'] = [tmp['StreetName'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePostType'] = [tmp['StreetNamePostType'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePostDirectional'] = [tmp['StreetNamePostDirectional'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'PlaceName'] = [tmp['PlaceName'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StateName'] = [tmp['StateName'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'ZipCode'] = [tmp['ZipCode'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]

        # OMOP_location.loc[ind, 'AddressNumber'] = obj[0]['AddressNumber']
        # OMOP_location.loc[ind, 'StreetNamePreDirectional'] = obj[0]['StreetNamePreDirectional']
        # OMOP_location.loc[ind, 'StreetNamePreType'] = obj[0]['StreetNamePreType']
        # OMOP_location.loc[ind, 'StreetName'] = obj[0]['StreetName']
        # OMOP_location.loc[ind, 'StreetNamePostType'] = obj[0]['StreetNamePostType']
        # OMOP_location.loc[ind, 'StreetNamePostDirectional'] = obj[0]['StreetNamePostDirectional']
        # OMOP_location.loc[ind, 'PlaceName']=obj[0]['PlaceName']
        # OMOP_location.loc[ind, 'StateName']=obj[0]['StateName']
        # OMOP_location.loc[ind, 'ZipCode']=obj[0]['ZipCode']

        repo.append(tmp)

        # StreetNamePreDirectional = tmp['StreetNamePreDirectional'].values[0]
        # if not StreetNamePreDirectional:
        #     OMOP_location.loc[ind, 'StreetNamePreDirectional'] = np.NaN
        # else:
        #     OMOP_location.loc[ind, 'StreetNamePreDirectional'] = StreetNamePreDirectional

    except:
        pass

CPU times: total: 33.4 s
Wall time: 1min 5s


In [9]:
%%time

# concatenate above list to a single dataframe and only keep address components for Nominatim
nominatim_components = pd.concat(repo)
nominatim_components[['location_id','AddressNumber', 'StreetNamePreDirectional', 'StreetNamePreType', 'StreetName', 'StreetNamePostType','StreetNamePostDirectional',
                      'PlaceName', 'county', 'StateName', 'ZipCode', 'location_source_value']]

CPU times: total: 13.6 s
Wall time: 35.5 s


,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,PlaceName,county,StateName,ZipCode,location_source_value
0,1,3210,NaN,NaN,SOUTHWESTERN,BLVD,NaN,ORCHARD PARK,Erie County,NY,14127,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127"
1,2,562,NaN,NaN,WINGATE,DR,NaN,EAST MEADOW,Nassau County,NY,11554,"562 WINGATE DR, EAST MEADOW, NY 11554"
2,3,8,NaN,NaN,ZOAR,AVE,NaN,ALBANY,Albany County,NY,12209,"8 ZOAR AVE, ALBANY, NY 12209"
3,4,15402,NaN,NaN,41ST,AVE,NaN,FLUSHING,Queens County,NY,11354,"15402 41ST AVE, FLUSHING, NY 11354"
4,5,5110,NaN,NaN,19TH,AVE,NaN,BROOKLYN,Kings County,NY,11204,"5110 19TH AVE, BROOKLYN, NY 11204"
...,...,...,...,...,...,...,...,...,...,...,...,...
40412,40413,5000,N,NaN,BOWES,RD,NaN,TUCSON,Pima County,AZ,85749,"5000 N BOWES RD, TUCSON, AZ 85749"
40413,40414,1303,NaN,NaN,PASEO DEL CANON,NaN,EAST,TAOS,Taos County,NM,87571,"1303 PASEO DEL CANON EAST, TAOS, NM 87571"
40414,40415,12606,E,NaN,MAIN,ST,NaN,MAYER,Yavapai County,AZ,86333,"12606 E MAIN ST, MAYER, AZ 86333"
40415,40416,4300,NaN,NaN,BLAKE,RD,SW,ALBUQUERQUE,Bernalillo County,NM,87121,"4300 BLAKE RD SW, ALBUQUERQUE, NM 87121"


In [10]:
# keep only columns necessary to merge to omop_county_zcta_zip_match
nominatim_keep_columns = nominatim_components[['location_id','AddressNumber', 'StreetNamePreDirectional', 'StreetNamePreType', 'StreetName', 'StreetNamePostType','StreetNamePostDirectional','county',]]
nominatim_keep_columns

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county
0,1,3210,NaN,NaN,SOUTHWESTERN,BLVD,NaN,Erie County
1,2,562,NaN,NaN,WINGATE,DR,NaN,Nassau County
2,3,8,NaN,NaN,ZOAR,AVE,NaN,Albany County
3,4,15402,NaN,NaN,41ST,AVE,NaN,Queens County
4,5,5110,NaN,NaN,19TH,AVE,NaN,Kings County
...,...,...,...,...,...,...,...,...
40412,40413,5000,N,NaN,BOWES,RD,NaN,Pima County
40413,40414,1303,NaN,NaN,PASEO DEL CANON,NaN,EAST,Taos County
40414,40415,12606,E,NaN,MAIN,ST,NaN,Yavapai County
40415,40416,4300,NaN,NaN,BLAKE,RD,SW,Bernalillo County


In [13]:
# create empty dataframe
nominatim_location = pd.DataFrame(columns=['location_id', 'address_1', 'address_2', 'city', 'state', 'zip', 'state_abbr', 'location_source_value','latitude','longitude', 'geometry'])

# fill in address and new location_id from omop_county_zcta_zip_match 
nominatim_location['location_source_value'] = omop_county_zcta_zip_match.location_source_value
nominatim_location['location_id'] = nominatim_location.index+1
# nominatim_location['county'] = omop_county_zcta_zip_match.county

# copy over the same columns from omop_county_zcta_zip_match
nominatim_location.address_1 = omop_county_zcta_zip_match.address_1
nominatim_location.address_2 = omop_county_zcta_zip_match.address_2
nominatim_location.city = omop_county_zcta_zip_match.city
nominatim_location.state = omop_county_zcta_zip_match.state
nominatim_location.zip = omop_county_zcta_zip_match.zip
nominatim_location.state_abbr = omop_county_zcta_zip_match.state_abbr
nominatim_location.latitude = omop_county_zcta_zip_match.latitude
nominatim_location.longitude = omop_county_zcta_zip_match.longitude
nominatim_location.geometry = omop_county_zcta_zip_match.geometry
nominatim_location

,location_id,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry
0,1,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358)
1,2,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788)
2,3,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181)
3,4,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896)
4,5,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931)
...,...,...,...,...,...,...,...,...,...,...,...
40412,40413,5000 N Bowes Rd,NaN,Tucson,Arizona,85749,AZ,"5000 N BOWES RD, TUCSON, AZ 85749",32.298797,-110.788349,POINT (-110.7883488373378 32.29879689494836)
40413,40414,1303 Paseo Del Canon East,NaN,Taos,New Mexico,87571,NM,"1303 PASEO DEL CANON EAST, TAOS, NM 87571",36.375292,-105.568573,POINT (-105.568573404848 36.37529177977968)
40414,40415,12606 E Main St,NaN,Mayer,Arizona,86333,AZ,"12606 E MAIN ST, MAYER, AZ 86333",34.346322,-112.157189,POINT (-112.1571889686942 34.34632218671697)
40415,40416,4300 Blake Rd Sw,NaN,Albuquerque,New Mexico,87121,NM,"4300 BLAKE RD SW, ALBUQUERQUE, NM 87121",35.037163,-106.723264,POINT (-106.7232635088362 35.03716265606487)


In [23]:
# merge omop_county_zcta_zip_match addresses to the ones that parsed fully from nominatim_keep_columns_merge_county
nominatim_keep_columns_merge_county = nominatim_keep_columns.merge(nominatim_location, how='left', on='location_id')
print(nominatim_keep_columns_merge_county.dtypes)
nominatim_keep_columns_merge_county

location_id                    int64
AddressNumber                 object
StreetNamePreDirectional      object
StreetNamePreType             object
StreetName                    object
StreetNamePostType            object
StreetNamePostDirectional     object
county                        object
address_1                     object
address_2                     object
city                          object
state                         object
zip                            int64
state_abbr                    object
location_source_value         object
latitude                     float64
longitude                    float64
geometry                      object
dtype: object


,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry
0,1,3210,NaN,NaN,SOUTHWESTERN,BLVD,NaN,Erie County,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358)
1,2,562,NaN,NaN,WINGATE,DR,NaN,Nassau County,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788)
2,3,8,NaN,NaN,ZOAR,AVE,NaN,Albany County,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181)
3,4,15402,NaN,NaN,41ST,AVE,NaN,Queens County,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896)
4,5,5110,NaN,NaN,19TH,AVE,NaN,Kings County,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40349,40413,5000,N,NaN,BOWES,RD,NaN,Pima County,5000 N Bowes Rd,NaN,Tucson,Arizona,85749,AZ,"5000 N BOWES RD, TUCSON, AZ 85749",32.298797,-110.788349,POINT (-110.7883488373378 32.29879689494836)
40350,40414,1303,NaN,NaN,PASEO DEL CANON,NaN,EAST,Taos County,1303 Paseo Del Canon East,NaN,Taos,New Mexico,87571,NM,"1303 PASEO DEL CANON EAST, TAOS, NM 87571",36.375292,-105.568573,POINT (-105.568573404848 36.37529177977968)
40351,40415,12606,E,NaN,MAIN,ST,NaN,Yavapai County,12606 E Main St,NaN,Mayer,Arizona,86333,AZ,"12606 E MAIN ST, MAYER, AZ 86333",34.346322,-112.157189,POINT (-112.1571889686942 34.34632218671697)
40352,40416,4300,NaN,NaN,BLAKE,RD,SW,Bernalillo County,4300 Blake Rd Sw,NaN,Albuquerque,New Mexico,87121,NM,"4300 BLAKE RD SW, ALBUQUERQUE, NM 87121",35.037163,-106.723264,POINT (-106.7232635088362 35.03716265606487)


In [24]:
# replace cardinal direction abbreviations with full string and capitalize only first letter
nominatim_keep_columns_merge_county['StreetNamePreDirectional'] = nominatim_keep_columns_merge_county.StreetNamePreDirectional.apply(lambda x: np.NaN if x == np.NaN else cardinal_direction_lambda(x, cardinal_directions_to_full))
nominatim_keep_columns_merge_county['StreetNamePreDirectional'] = nominatim_keep_columns_merge_county.StreetNamePreDirectional.apply(lambda x: str(x).title())

# replace cardinal direction abbreviations with full string and capitalize only first letter
nominatim_keep_columns_merge_county['StreetNamePostDirectional'] = nominatim_keep_columns_merge_county.StreetNamePostDirectional.apply(lambda x: np.NaN if x == np.NaN else cardinal_direction_lambda(x, cardinal_directions_to_full))
nominatim_keep_columns_merge_county['StreetNamePostDirectional'] = nominatim_keep_columns_merge_county.StreetNamePostDirectional.apply(lambda x: str(x).title())

# capitalize only first letters
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).title())
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).title())

# replace street post abbreviations with full string
nominatim_keep_columns_merge_county['StreetNamePreType'] = nominatim_keep_columns_merge_county.StreetNamePreType.apply(lambda x: street_suffix_lambda(str(x).lower(), street_suffix_to_full))
nominatim_keep_columns_merge_county['StreetNamePostType'] = nominatim_keep_columns_merge_county.StreetNamePostType.apply(lambda x: street_suffix_lambda(str(x).lower(), street_suffix_to_full))

# remove ending whitespaces
nominatim_keep_columns_merge_county['AddressNumber'] = nominatim_keep_columns_merge_county.AddressNumber.apply(lambda x: str(x).rstrip())
nominatim_keep_columns_merge_county

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry
0,1,3210,Nan,NaN,Southwestern,Boulevard,Nan,Erie County,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358)
1,2,562,Nan,NaN,Wingate,Drive,Nan,Nassau County,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788)
2,3,8,Nan,NaN,Zoar,Avenue,Nan,Albany County,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181)
3,4,15402,Nan,NaN,41St,Avenue,Nan,Queens County,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896)
4,5,5110,Nan,NaN,19Th,Avenue,Nan,Kings County,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40349,40413,5000,North,NaN,Bowes,Road,Nan,Pima County,5000 N Bowes Rd,NaN,Tucson,Arizona,85749,AZ,"5000 N BOWES RD, TUCSON, AZ 85749",32.298797,-110.788349,POINT (-110.7883488373378 32.29879689494836)
40350,40414,1303,Nan,NaN,Paseo Del Canon,NaN,East,Taos County,1303 Paseo Del Canon East,NaN,Taos,New Mexico,87571,NM,"1303 PASEO DEL CANON EAST, TAOS, NM 87571",36.375292,-105.568573,POINT (-105.568573404848 36.37529177977968)
40351,40415,12606,East,NaN,Main,Street,Nan,Yavapai County,12606 E Main St,NaN,Mayer,Arizona,86333,AZ,"12606 E MAIN ST, MAYER, AZ 86333",34.346322,-112.157189,POINT (-112.1571889686942 34.34632218671697)
40352,40416,4300,Nan,NaN,Blake,Road,Southwest,Bernalillo County,4300 Blake Rd Sw,NaN,Albuquerque,New Mexico,87121,NM,"4300 BLAKE RD SW, ALBUQUERQUE, NM 87121",35.037163,-106.723264,POINT (-106.7232635088362 35.03716265606487)


In [ ]:
# postprocessing clean up
# convert zip to type string
nominatim_keep_columns_merge_county['zip'] = nominatim_keep_columns_merge_county.zip.astype(str)

# convert full string to ordinal numbers
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).lower())
# nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: replace_ordinal_numbers(str(x)))
# nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: make_ordinal(str(x)))
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).title() if not str(x)[0].isdigit() else str(x).lower())
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).replace("-", " "))

# capitalize first letter
nominatim_keep_columns_merge_county['StreetNamePreType'] = nominatim_keep_columns_merge_county.StreetNamePreType.apply(lambda x: np.NaN if x == np.NaN else str(x).title())
nominatim_keep_columns_merge_county['StreetNamePostType'] = nominatim_keep_columns_merge_county.StreetNamePostType.apply(lambda x: np.NaN if x == np.NaN else str(x).title())

# join Nominatim components to a single string of AddressNumber, StreetNamePreDirectional + StreetNamePreType + StreetName + StreetNamePostType, city, county, state, zip, United States
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county['AddressNumber'] + ',' + \
                                                            nominatim_keep_columns_merge_county[['StreetNamePreDirectional', 'StreetNamePreType', 'StreetName', 'StreetNamePostType', 'StreetNamePostDirectional']].fillna('').agg(' '.join, axis=1) + ', '\
                                                            + nominatim_keep_columns_merge_county[['city', 'county', 'state', 'zip']].fillna('').agg(', '.join, axis=1) + ', United States'

# remove double whitespaces
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county.Nominatim_address.apply(lambda x: x.replace("  ", " "))

# replace string "nan" with np.NaN
nominatim_keep_columns_merge_county['StreetNamePreDirectional'] = nominatim_keep_columns_merge_county.StreetNamePreDirectional.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)
nominatim_keep_columns_merge_county['StreetNamePostDirectional'] = nominatim_keep_columns_merge_county.StreetNamePostDirectional.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)

nominatim_keep_columns_merge_county['StreetNamePreType'] = nominatim_keep_columns_merge_county.StreetNamePreType.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)
nominatim_keep_columns_merge_county['StreetNamePostType'] = nominatim_keep_columns_merge_county.StreetNamePostType.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)

# replace "Nan" string from directionals
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county.Nominatim_address.apply(lambda x: str(x).replace('Nan', ''))

# replace adjacent commas with one comma
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county.Nominatim_address.apply(lambda x: str(x).replace(', ,', ', '))
nominatim_keep_columns_merge_county                                                                                                                      

In [28]:
nominatim_keep_columns_merge_county.iloc[4].Nominatim_address

'5110,  19th Avenue, Brooklyn, Kings County, New York, 11204, United States'

In [29]:
nominatim_keep_columns_merge_county_parsed_path = os.path.join(abs_path, 'output', 'nominatim_keep_columns_merge_county_parsed.csv')
nominatim_keep_columns_merge_county.to_csv(nominatim_keep_columns_merge_county_parsed_path, index=False)

# Random Sampling Nominatim Addresses
* 10 per state plus US territories

In [4]:
nominatim_keep_columns_merge_county_parsed_path = os.path.join(abs_path, 'output', 'nominatim_keep_columns_merge_county_parsed.csv')
nominatim_keep_columns_merge_county = pd.read_csv(nominatim_keep_columns_merge_county_parsed_path)
nominatim_keep_columns_merge_county

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry,Nominatim_address
0,1,3210,NaN,NaN,Southwestern,Boulevard,NaN,Erie County,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358),"3210, Southwestern Boulevard, Orchard Park, E..."
1,2,562,NaN,NaN,Wingate,Drive,NaN,Nassau County,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788),"562, Wingate Drive, East Meadow, Nassau Count..."
2,3,8,NaN,NaN,Zoar,Avenue,NaN,Albany County,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181),"8, Zoar Avenue, Albany, Albany County, New Yo..."
3,4,15402,NaN,NaN,41st,Avenue,NaN,Queens County,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896),"15402, 41st Avenue, Flushing, Queens County, ..."
4,5,5110,NaN,NaN,19th,Avenue,NaN,Kings County,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931),"5110, 19th Avenue, Brooklyn, Kings County, Ne..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40349,40413,5000,North,NaN,Bowes,Road,NaN,Pima County,5000 N Bowes Rd,NaN,Tucson,Arizona,85749,AZ,"5000 N BOWES RD, TUCSON, AZ 85749",32.298797,-110.788349,POINT (-110.7883488373378 32.29879689494836),"5000,North Bowes Road, Tucson, Pima County, A..."
40350,40414,1303,NaN,NaN,Paseo Del Canon,NaN,East,Taos County,1303 Paseo Del Canon East,NaN,Taos,New Mexico,87571,NM,"1303 PASEO DEL CANON EAST, TAOS, NM 87571",36.375292,-105.568573,POINT (-105.568573404848 36.37529177977968),"1303, Paseo Del Canon , Taos, Taos County, Ne..."
40351,40415,12606,East,NaN,Main,Street,NaN,Yavapai County,12606 E Main St,NaN,Mayer,Arizona,86333,AZ,"12606 E MAIN ST, MAYER, AZ 86333",34.346322,-112.157189,POINT (-112.1571889686942 34.34632218671697),"12606,East Main Street, Mayer, Yavapai County..."
40352,40416,4300,NaN,NaN,Blake,Road,Southwest,Bernalillo County,4300 Blake Rd Sw,NaN,Albuquerque,New Mexico,87121,NM,"4300 BLAKE RD SW, ALBUQUERQUE, NM 87121",35.037163,-106.723264,POINT (-106.7232635088362 35.03716265606487),"4300, Blake Road, Albuquerque, Bernalillo Cou..."


In [7]:
territories_drop = ['MP', 'VI', 'PW', 'Of', 'PR', 'GU']

dropped_territories = nominatim_keep_columns_merge_county.loc[nominatim_keep_columns_merge_county.state_abbr.isin(territories_drop)]
nominatim_keep_columns_merge_county_drop = nominatim_keep_columns_merge_county.loc[~nominatim_keep_columns_merge_county.state_abbr.isin(territories_drop)]

print(nominatim_keep_columns_merge_county_drop.shape)

(40354, 19)


In [17]:
nominatim_keep_columns_merge_county.state_abbr.unique()

array(['NY', 'SD', 'NE', 'ND', 'UT', 'NV', 'NM', 'MT', 'CO', 'ID', 'AZ',
       'OR', 'WY', 'MO', 'WA', 'FL', 'KY', 'MD', 'MI', 'AL', 'OH', 'TX',
       'CA', 'AR', 'TN', 'WI', 'IL', 'VA', 'PA', 'DE', 'IN', 'GA', 'WV',
       'MN', 'KS', 'IA', 'SC', 'OK', 'NC', 'MS', 'HI'], dtype=object)

In [15]:
nominatim_sample = nominatim_keep_columns_merge_county_drop.groupby('state_abbr').apply(lambda x: x.sample(n=13)).reset_index(drop = True)
nominatim_sample

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry,Nominatim_address
0,33497,3616,NaN,NaN,Holmes,Avenue,Northwest,Madison County,3616 Holmes Ave Nw,NaN,Huntsville,Alabama,35816,AL,"3616 HOLMES AVE NW, HUNTSVILLE, AL 35816",34.727989,-86.622204,POINT (-86.62220359781466 34.72798939318264),"3616, Holmes Avenue, Huntsville, Madison Coun..."
1,16780,3,NaN,NaN,Water,Street,NaN,Wilcox County,3 Water Street,NaN,Camden,Alabama,36726,AL,"3 Water Street, Camden, Alabama 36726",31.990317,-87.288132,POINT (-87.288132 31.990317),"3, Water Street, Camden, Wilcox County, Alaba..."
2,33447,751A,NaN,NaN,Academy,Drive,NaN,Jefferson County,751A Academy Dr,NaN,Bessemer,Alabama,35022,AL,"751A ACADEMY DR, BESSEMER, AL 35022",33.360897,-86.997065,POINT (-86.99706549398968 33.36089687137257),"751A, Academy Drive, Bessemer, Jefferson Coun..."
3,22962,809,NaN,NaN,University,Boulevard,East,Tuscaloosa County,809 University Blvd East,4TH FLOOR,Tuscaloosa,Alabama,35401,AL,"809 UNIVERSITY BLVD EAST, 4TH FLOOR, TUSCALOOS...",33.205174,-87.526364,POINT (-87.52636438 33.2051744500001),"809, University Boulevard, Tuscaloosa, Tuscal..."
4,33413,15048,NaN,Us Highway,431,NaN,NaN,Chambers County,15048 Us Highway 431,NaN,Lafayette,Alabama,36862,AL,"15048 US HIGHWAY 431, LAFAYETTE, AL 36862",32.873045,-85.406553,POINT (-85.40655254573336 32.87304464022434),"15048, Us Highway 431 , Lafayette, Chambers Co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,33330,1100,NaN,NaN,Bruce,Lane,NaN,Natrona County,1100 Bruce Lane,NaN,Casper,Wyoming,82601,WY,"1100 BRUCE LANE, CASPER, WY 82601",42.863028,-106.345419,POINT (-106.3454187986559 42.86302796718916),"1100, Bruce Lane, Casper, Natrona County, Wyo..."
529,27415,321,West,NaN,Birch,Street,NaN,Converse County,321 W Birch St,NaN,Glenrock,Wyoming,82637,WY,"321 W Birch St, Glenrock, WY 82637",42.861254,-105.871882,POINT (-105.8718823003468 42.86125380010095),"321,West Birch Street, Glenrock, Converse Cou..."
530,4000,304,North,NaN,Cheyenne,Avenue,NaN,Crook County,304 N Cheyenne Ave,NaN,Moorcroft,Wyoming,82721,WY,"304 N CHEYENNE AVE, MOORCROFT, WY 82721",44.265919,-104.953240,POINT (-104.953239999962 44.26591899968666),"304,North Cheyenne Avenue, Moorcroft, Crook C..."
531,8064,1876,South,NaN,Sheridan,Avenue,NaN,Sheridan County,1876 S Sheridan Ave,NaN,Sheridan,Wyoming,82801,WY,"1876 S SHERIDAN AVE, SHERIDAN, WY 82801",44.777627,-106.947468,POINT (-106.9474680000906 44.77762699988288),"1876,South Sheridan Avenue, Sheridan, Sherida..."


In [16]:
nominatim_sample_path = os.path.join(abs_path, 'output', 'nominatim_sample.csv')
nominatim_sample.to_csv(nominatim_sample_path, index=False)

# Nominatim Parsing
* With `omop_county_zcta_match`

In [38]:
omop_county_zcta_path = os.path.join(abs_path, 'output', 'omop_county_zcta.csv')
omop_county_zcta.to_csv(omop_county_zcta_path, index=False)
omop_county_zcta_match = omop_county_zcta.loc[omop_county_zcta.county_match == 1]

In [39]:
omop_county_zcta_match = omop_county_zcta_match.reset_index()
omop_county_zcta_match['location_id'] = omop_county_zcta_match.index+1
omop_county_zcta_match = omop_county_zcta_match.drop(columns=['index', 'Location_id'])
omop_county_zcta_match = omop_county_zcta_match.rename(columns={'zip_left': 'zip'})
omop_county_zcta_match

,address_1,address_2,city,state,zip,county,location_source_value,latitude,longitude,address_type,state_abbr,flag,geometry,ZCTA5CE20,GEOID20,NAMELSAD_COUNTY_20,county_match,location_id
0,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,Erie County,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-78.74781599997843 42.79853499984358),14127.0,14127.0,Erie County,1,1
1,562 Wingate Dr,NaN,East Meadow,New York,11554,Nassau County,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.52838899973285 40.71011299966788),11554.0,11554.0,Nassau County,1,2
2,8 Zoar Ave,NaN,Albany,New York,12209,Albany County,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.78899100027446 42.63906699972181),12209.0,12209.0,Albany County,1,3
3,15402 41St Ave,NaN,Flushing,New York,11354,Queens County,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.81094400044168 40.76300300008896),11354.0,11354.0,Queens County,1,4
4,5110 19Th Ave,NaN,Brooklyn,New York,11204,Kings County,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,Street Address,NY,SUCCESSFUL ADDRESS,POINT (-73.98001299964284 40.62562299970931),11204.0,11204.0,Kings County,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65692,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,Gwinnett County,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.09517421962246 34.04172694900986),30024.0,30024.0,Gwinnett County,1,65693
65693,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,Gwinnett County,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-84.10131848955332 33.96679709299809),30096.0,30096.0,Gwinnett County,1,65694
65694,3685 Braselton Highway,NaN,Dacula,Georgia,30019,Gwinnett County,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,Street Address,GA,SUCCESSFUL ADDRESS,POINT (-83.90221543184028 34.06883234729177),30019.0,30019.0,Gwinnett County,1,65695
65695,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,Fayette County,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,Street Address,KY,SUCCESSFUL ADDRESS,POINT (-84.49410603994914 37.99650802456039),40502.0,40502.0,Fayette County,1,65696


In [40]:
%%time

# run `usaddress` parsing with base component names and append to list
repo = []
for ind, each in omop_county_zcta_match.iterrows():
    try:
        obj = usaddress.tag(each.location_source_value)
        tmp = pd.DataFrame(obj[0], columns=obj[0].keys(), index=[ind])
        tmp['Address_type'] = obj[1]
        tmp['county'] = each.county
        tmp['location_source_value'] = each.location_source_value
        tmp['location_id'] = each.location_id

        # development
        # OMOP_location.loc[ind, 'AddressNumber'] = [tmp['AddressNumber'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePreDirectional'] = [tmp['StreetNamePreDirectional'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePreType'] = [tmp['StreetNamePreType'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetName'] = [tmp['StreetName'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePostType'] = [tmp['StreetNamePostType'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StreetNamePostDirectional'] = [tmp['StreetNamePostDirectional'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'PlaceName'] = [tmp['PlaceName'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'StateName'] = [tmp['StateName'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]
        # OMOP_location.loc[ind, 'ZipCode'] = [tmp['ZipCode'].values[0] if len(tmp['AddressNumber'].values[0]) > 0 else np.NaN][0]

        # OMOP_location.loc[ind, 'AddressNumber'] = obj[0]['AddressNumber']
        # OMOP_location.loc[ind, 'StreetNamePreDirectional'] = obj[0]['StreetNamePreDirectional']
        # OMOP_location.loc[ind, 'StreetNamePreType'] = obj[0]['StreetNamePreType']
        # OMOP_location.loc[ind, 'StreetName'] = obj[0]['StreetName']
        # OMOP_location.loc[ind, 'StreetNamePostType'] = obj[0]['StreetNamePostType']
        # OMOP_location.loc[ind, 'StreetNamePostDirectional'] = obj[0]['StreetNamePostDirectional']
        # OMOP_location.loc[ind, 'PlaceName']=obj[0]['PlaceName']
        # OMOP_location.loc[ind, 'StateName']=obj[0]['StateName']
        # OMOP_location.loc[ind, 'ZipCode']=obj[0]['ZipCode']

        repo.append(tmp)

        # StreetNamePreDirectional = tmp['StreetNamePreDirectional'].values[0]
        # if not StreetNamePreDirectional:
        #     OMOP_location.loc[ind, 'StreetNamePreDirectional'] = np.NaN
        # else:
        #     OMOP_location.loc[ind, 'StreetNamePreDirectional'] = StreetNamePreDirectional

    except:
        pass

CPU times: total: 1min 14s
Wall time: 1min 34s


In [41]:
%%time

# concatenate above list to a single dataframe and only keep address components for Nominatim
nominatim_components = pd.concat(repo)
nominatim_components[['location_id','AddressNumber', 'StreetNamePreDirectional', 'StreetNamePreType', 'StreetName', 'StreetNamePostType','StreetNamePostDirectional',
                      'PlaceName', 'county', 'StateName', 'ZipCode', 'location_source_value']]

CPU times: total: 41.3 s
Wall time: 51.3 s


,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,PlaceName,county,StateName,ZipCode,location_source_value
0,1,3210,NaN,NaN,SOUTHWESTERN,BLVD,NaN,ORCHARD PARK,Erie County,NY,14127,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127"
1,2,562,NaN,NaN,WINGATE,DR,NaN,EAST MEADOW,Nassau County,NY,11554,"562 WINGATE DR, EAST MEADOW, NY 11554"
2,3,8,NaN,NaN,ZOAR,AVE,NaN,ALBANY,Albany County,NY,12209,"8 ZOAR AVE, ALBANY, NY 12209"
3,4,15402,NaN,NaN,41ST,AVE,NaN,FLUSHING,Queens County,NY,11354,"15402 41ST AVE, FLUSHING, NY 11354"
4,5,5110,NaN,NaN,19TH,AVE,NaN,BROOKLYN,Kings County,NY,11204,"5110 19TH AVE, BROOKLYN, NY 11204"
...,...,...,...,...,...,...,...,...,...,...,...,...
65692,65693,1300,NaN,NaN,PEACHTREE INDUSTRIAL,BOULEVARD,NaN,SUWANEE,Gwinnett County,GA,30024,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ..."
65693,65694,2660,NaN,NaN,SATELLITE,BOULEVARD,NORTHWEST,DULUTH,Gwinnett County,GA,30096,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA..."
65694,65695,3685,NaN,NaN,BRASELTON,HIGHWAY,NaN,DACULA,Gwinnett County,GA,30019,"3685 BRASELTON HIGHWAY, DACULA, GA 30019"
65695,65696,1055,NaN,NaN,DOVE RUN,ROAD,NaN,LEXINGTON,Fayette County,KY,40502,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502"


In [42]:
# keep only columns necessary to merge to omop_county_zcta_zip_match
nominatim_keep_columns = nominatim_components[['location_id','AddressNumber', 'StreetNamePreDirectional', 'StreetNamePreType', 'StreetName', 'StreetNamePostType','StreetNamePostDirectional','county', 'ZipCode']]
nominatim_keep_columns

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,ZipCode
0,1,3210,NaN,NaN,SOUTHWESTERN,BLVD,NaN,Erie County,14127
1,2,562,NaN,NaN,WINGATE,DR,NaN,Nassau County,11554
2,3,8,NaN,NaN,ZOAR,AVE,NaN,Albany County,12209
3,4,15402,NaN,NaN,41ST,AVE,NaN,Queens County,11354
4,5,5110,NaN,NaN,19TH,AVE,NaN,Kings County,11204
...,...,...,...,...,...,...,...,...,...
65692,65693,1300,NaN,NaN,PEACHTREE INDUSTRIAL,BOULEVARD,NaN,Gwinnett County,30024
65693,65694,2660,NaN,NaN,SATELLITE,BOULEVARD,NORTHWEST,Gwinnett County,30096
65694,65695,3685,NaN,NaN,BRASELTON,HIGHWAY,NaN,Gwinnett County,30019
65695,65696,1055,NaN,NaN,DOVE RUN,ROAD,NaN,Fayette County,40502


In [43]:
# create empty dataframe
nominatim_location = pd.DataFrame(columns=['location_id', 'address_1', 'address_2', 'city', 'state', 'zip', 'state_abbr', 'location_source_value','latitude','longitude', 'geometry'])

# omop_county_zcta_match = omop_county_zcta_match.reset_index()
# fill in address and new location_id from omop_county_zcta_zip_match 
nominatim_location['location_source_value'] = omop_county_zcta_match.location_source_value
nominatim_location['location_id'] = nominatim_location.index+1
# nominatim_location['county'] = omop_county_zcta_zip_match.county

# copy over the same columns from omop_county_zcta_zip_match
nominatim_location.address_1 = omop_county_zcta_match.address_1
nominatim_location.address_2 = omop_county_zcta_match.address_2
nominatim_location.city = omop_county_zcta_match.city
nominatim_location.state = omop_county_zcta_match.state
nominatim_location.zip = omop_county_zcta_match.zip
nominatim_location.state_abbr = omop_county_zcta_match.state_abbr
nominatim_location.latitude = omop_county_zcta_match.latitude
nominatim_location.longitude = omop_county_zcta_match.longitude
nominatim_location.geometry = omop_county_zcta_match.geometry
nominatim_location

,location_id,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry
0,1,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358)
1,2,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788)
2,3,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181)
3,4,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896)
4,5,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931)
...,...,...,...,...,...,...,...,...,...,...,...
65692,65693,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,GA,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,POINT (-84.09517421962246 34.04172694900986)
65693,65694,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,GA,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,POINT (-84.10131848955332 33.96679709299809)
65694,65695,3685 Braselton Highway,NaN,Dacula,Georgia,30019,GA,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,POINT (-83.90221543184028 34.06883234729177)
65695,65696,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,KY,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,POINT (-84.49410603994914 37.99650802456039)


In [44]:
# merge omop_county_zcta_zip_match addresses to the ones that parsed fully from nominatim_keep_columns_merge_county
nominatim_keep_columns_merge_county = nominatim_keep_columns.merge(nominatim_location, how='left', on='location_id')
print(nominatim_keep_columns_merge_county.dtypes)
nominatim_keep_columns_merge_county

location_id                    int64
AddressNumber                 object
StreetNamePreDirectional      object
StreetNamePreType             object
StreetName                    object
StreetNamePostType            object
StreetNamePostDirectional     object
county                        object
ZipCode                       object
address_1                     object
address_2                     object
city                          object
state                         object
zip                           object
state_abbr                    object
location_source_value         object
latitude                     float64
longitude                    float64
geometry                      object
dtype: object


,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,ZipCode,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry
0,1,3210,NaN,NaN,SOUTHWESTERN,BLVD,NaN,Erie County,14127,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358)
1,2,562,NaN,NaN,WINGATE,DR,NaN,Nassau County,11554,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788)
2,3,8,NaN,NaN,ZOAR,AVE,NaN,Albany County,12209,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181)
3,4,15402,NaN,NaN,41ST,AVE,NaN,Queens County,11354,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896)
4,5,5110,NaN,NaN,19TH,AVE,NaN,Kings County,11204,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65504,65693,1300,NaN,NaN,PEACHTREE INDUSTRIAL,BOULEVARD,NaN,Gwinnett County,30024,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,GA,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,POINT (-84.09517421962246 34.04172694900986)
65505,65694,2660,NaN,NaN,SATELLITE,BOULEVARD,NORTHWEST,Gwinnett County,30096,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,GA,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,POINT (-84.10131848955332 33.96679709299809)
65506,65695,3685,NaN,NaN,BRASELTON,HIGHWAY,NaN,Gwinnett County,30019,3685 Braselton Highway,NaN,Dacula,Georgia,30019,GA,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,POINT (-83.90221543184028 34.06883234729177)
65507,65696,1055,NaN,NaN,DOVE RUN,ROAD,NaN,Fayette County,40502,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,KY,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,POINT (-84.49410603994914 37.99650802456039)


In [45]:
# replace cardinal direction abbreviations with full string and capitalize only first letter
nominatim_keep_columns_merge_county['StreetNamePreDirectional'] = nominatim_keep_columns_merge_county.StreetNamePreDirectional.apply(lambda x: np.NaN if x == np.NaN else cardinal_direction_lambda(x, cardinal_directions_to_full))
nominatim_keep_columns_merge_county['StreetNamePreDirectional'] = nominatim_keep_columns_merge_county.StreetNamePreDirectional.apply(lambda x: str(x).title())

# replace cardinal direction abbreviations with full string and capitalize only first letter
nominatim_keep_columns_merge_county['StreetNamePostDirectional'] = nominatim_keep_columns_merge_county.StreetNamePostDirectional.apply(lambda x: np.NaN if x == np.NaN else cardinal_direction_lambda(x, cardinal_directions_to_full))
nominatim_keep_columns_merge_county['StreetNamePostDirectional'] = nominatim_keep_columns_merge_county.StreetNamePostDirectional.apply(lambda x: str(x).title())

# capitalize only first letters
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).title())
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).title())

# replace street post abbreviations with full string
nominatim_keep_columns_merge_county['StreetNamePreType'] = nominatim_keep_columns_merge_county.StreetNamePreType.apply(lambda x: street_suffix_lambda(str(x).lower(), street_suffix_to_full))
nominatim_keep_columns_merge_county['StreetNamePostType'] = nominatim_keep_columns_merge_county.StreetNamePostType.apply(lambda x: street_suffix_lambda(str(x).lower(), street_suffix_to_full))

# remove ending whitespaces
nominatim_keep_columns_merge_county['AddressNumber'] = nominatim_keep_columns_merge_county.AddressNumber.apply(lambda x: str(x).rstrip())
nominatim_keep_columns_merge_county

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,ZipCode,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry
0,1,3210,Nan,NaN,Southwestern,Boulevard,Nan,Erie County,14127,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358)
1,2,562,Nan,NaN,Wingate,Drive,Nan,Nassau County,11554,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788)
2,3,8,Nan,NaN,Zoar,Avenue,Nan,Albany County,12209,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181)
3,4,15402,Nan,NaN,41St,Avenue,Nan,Queens County,11354,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896)
4,5,5110,Nan,NaN,19Th,Avenue,Nan,Kings County,11204,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65504,65693,1300,Nan,NaN,Peachtree Industrial,boulevard,Nan,Gwinnett County,30024,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,GA,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,POINT (-84.09517421962246 34.04172694900986)
65505,65694,2660,Nan,NaN,Satellite,boulevard,Northwest,Gwinnett County,30096,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,GA,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,POINT (-84.10131848955332 33.96679709299809)
65506,65695,3685,Nan,NaN,Braselton,highway,Nan,Gwinnett County,30019,3685 Braselton Highway,NaN,Dacula,Georgia,30019,GA,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,POINT (-83.90221543184028 34.06883234729177)
65507,65696,1055,Nan,NaN,Dove Run,road,Nan,Fayette County,40502,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,KY,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,POINT (-84.49410603994914 37.99650802456039)


In [46]:
# postprocessing clean up
# convert zip to type string
nominatim_keep_columns_merge_county['zip'] = nominatim_keep_columns_merge_county.zip.astype(str)

# convert full string to ordinal numbers
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).lower())
# nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: replace_ordinal_numbers(str(x)))
# nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: make_ordinal(str(x)))
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).title() if not str(x)[0].isdigit() else str(x).lower())
nominatim_keep_columns_merge_county['StreetName'] = nominatim_keep_columns_merge_county.StreetName.apply(lambda x: str(x).replace("-", " "))

# capitalize first letter
nominatim_keep_columns_merge_county['StreetNamePreType'] = nominatim_keep_columns_merge_county.StreetNamePreType.apply(lambda x: np.NaN if x == np.NaN else str(x).title())
nominatim_keep_columns_merge_county['StreetNamePostType'] = nominatim_keep_columns_merge_county.StreetNamePostType.apply(lambda x: np.NaN if x == np.NaN else str(x).title())

# join Nominatim components to a single string of AddressNumber, StreetNamePreDirectional + StreetNamePreType + StreetName + StreetNamePostType, city, county, state, zip, United States
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county['AddressNumber'] + ',' + \
                                                            nominatim_keep_columns_merge_county[['StreetNamePreDirectional', 'StreetNamePreType', 'StreetName', 'StreetNamePostType','StreetNamePostDirectional']].fillna('').agg(' '.join, axis=1) + ', '\
                                                            + nominatim_keep_columns_merge_county[['city', 'county', 'state', 'zip']].fillna('').agg(', '.join, axis=1) + ', United States'

# remove double whitespaces
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county.Nominatim_address.apply(lambda x: x.replace("  ", " "))

# replace string "nan" with np.NaN
nominatim_keep_columns_merge_county['StreetNamePreDirectional'] = nominatim_keep_columns_merge_county.StreetNamePreDirectional.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)
nominatim_keep_columns_merge_county['StreetNamePostDirectional'] = nominatim_keep_columns_merge_county.StreetNamePostDirectional.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)

nominatim_keep_columns_merge_county['StreetNamePreType'] = nominatim_keep_columns_merge_county.StreetNamePreType.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)
nominatim_keep_columns_merge_county['StreetNamePostType'] = nominatim_keep_columns_merge_county.StreetNamePostType.apply(lambda x: np.NaN if str(x) == "Nan" or str(x) == "nan" else x)

# replace "Nan" string from directionals
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county.Nominatim_address.apply(lambda x: str(x).replace('Nan', ''))

# replace adjacent commas with one comma
nominatim_keep_columns_merge_county['Nominatim_address'] = nominatim_keep_columns_merge_county.Nominatim_address.apply(lambda x: str(x).replace(', ,', ', '))
nominatim_keep_columns_merge_county                                                                                                                      

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,ZipCode,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry,Nominatim_address
0,1,3210,NaN,NaN,Southwestern,Boulevard,NaN,Erie County,14127,3210 Southwestern Blvd,NaN,Orchard Park,New York,14127,NY,"3210 SOUTHWESTERN BLVD, ORCHARD PARK, NY 14127",42.798535,-78.747816,POINT (-78.74781599997843 42.79853499984358),"3210, Southwestern Boulevard , Orchard Park, ..."
1,2,562,NaN,NaN,Wingate,Drive,NaN,Nassau County,11554,562 Wingate Dr,NaN,East Meadow,New York,11554,NY,"562 WINGATE DR, EAST MEADOW, NY 11554",40.710113,-73.528389,POINT (-73.52838899973285 40.71011299966788),"562, Wingate Drive , East Meadow, Nassau Coun..."
2,3,8,NaN,NaN,Zoar,Avenue,NaN,Albany County,12209,8 Zoar Ave,NaN,Albany,New York,12209,NY,"8 ZOAR AVE, ALBANY, NY 12209",42.639067,-73.788991,POINT (-73.78899100027446 42.63906699972181),"8, Zoar Avenue , Albany, Albany County, New Y..."
3,4,15402,NaN,NaN,41st,Avenue,NaN,Queens County,11354,15402 41St Ave,NaN,Flushing,New York,11354,NY,"15402 41ST AVE, FLUSHING, NY 11354",40.763003,-73.810944,POINT (-73.81094400044168 40.76300300008896),"15402, 41st Avenue , Flushing, Queens County,..."
4,5,5110,NaN,NaN,19th,Avenue,NaN,Kings County,11204,5110 19Th Ave,NaN,Brooklyn,New York,11204,NY,"5110 19TH AVE, BROOKLYN, NY 11204",40.625623,-73.980013,POINT (-73.98001299964284 40.62562299970931),"5110, 19th Avenue , Brooklyn, Kings County, N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65504,65693,1300,NaN,NaN,Peachtree Industrial,Boulevard,NaN,Gwinnett County,30024,1300 Peachtree Industrial Boulevard,NaN,Suwanee,Georgia,30024,GA,"1300 PEACHTREE INDUSTRIAL BOULEVARD, SUWANEE, ...",34.041727,-84.095174,POINT (-84.09517421962246 34.04172694900986),"1300, Peachtree Industrial Boulevard , Suwane..."
65505,65694,2660,NaN,NaN,Satellite,Boulevard,Northwest,Gwinnett County,30096,2660 Satellite Boulevard Northwest,NaN,Duluth,Georgia,30096,GA,"2660 SATELLITE BOULEVARD NORTHWEST, DULUTH, GA...",33.966797,-84.101318,POINT (-84.10131848955332 33.96679709299809),"2660, Satellite Boulevard Northwest, Duluth, ..."
65506,65695,3685,NaN,NaN,Braselton,Highway,NaN,Gwinnett County,30019,3685 Braselton Highway,NaN,Dacula,Georgia,30019,GA,"3685 BRASELTON HIGHWAY, DACULA, GA 30019",34.068832,-83.902215,POINT (-83.90221543184028 34.06883234729177),"3685, Braselton Highway , Dacula, Gwinnett Co..."
65507,65696,1055,NaN,NaN,Dove Run,Road,NaN,Fayette County,40502,1055 Dove Run Road,NaN,Lexington,Kentucky,40502,KY,"1055 DOVE RUN ROAD, LEXINGTON, KY 40502",37.996508,-84.494106,POINT (-84.49410603994914 37.99650802456039),"1055, Dove Run Road , Lexington, Fayette Coun..."


In [47]:
nominatim_keep_columns_merge_county_parsed_path = os.path.join(abs_path, 'output', 'nominatim_keep_columns_merge_county_parsed_spatialcrossmap.csv')
nominatim_keep_columns_merge_county.to_csv(nominatim_keep_columns_merge_county_parsed_path, index=False)

In [58]:
# random sampling
territories_drop = ['MP', 'VI', 'PW', 'Of', 'PR', 'GU', 'LA', 'NJ', 'CT', 'DC']

dropped_territories = nominatim_keep_columns_merge_county.loc[nominatim_keep_columns_merge_county.state_abbr.isin(territories_drop)]
nominatim_keep_columns_merge_county_drop = nominatim_keep_columns_merge_county.loc[~nominatim_keep_columns_merge_county.state_abbr.isin(territories_drop)]

# print(nominatim_keep_columns_merge_county_drop.shape)

df_sample = nominatim_keep_columns_merge_county_drop.groupby('state_abbr').apply(lambda x: x.sample(n=13)).reset_index(drop=True)
df_sample

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,ZipCode,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry,Nominatim_address
0,38362,5600,NaN,NaN,Debarr,Road,NaN,San Joaquin County,99504,5600 Debarr Road,NaN,Anchorage,Alaska,99504,AK,"5600 DEBARR ROAD, ANCHORAGE, AK 99504",38.112992,-121.303238,POINT (-121.3032381910243 38.11299177603096),"5600, Debarr Road , Anchorage, San Joaquin Co..."
1,38294,90,NaN,NaN,Sterling,Highway,NaN,Mendocino County,99603,90 Sterling Highway,NaN,Homer,Alaska,99603,AK,"90 STERLING HIGHWAY, HOMER, AK 99603",39.144519,-123.210948,POINT (-123.2109481534636 39.14451940539714),"90, Sterling Highway , Homer, Mendocino Count..."
2,38297,7731,East,NaN,Northern Lights,Boulevard,NaN,Contra Costa County,99504,7731 East Northern Lights Boulevard,NaN,Anchorage,Alaska,99504,AK,"7731 EAST NORTHERN LIGHTS BOULEVARD, ANCHORAGE...",37.898842,-122.320036,POINT (-122.3200358936853 37.89884185467251),"7731,East Northern Lights Boulevard , Anchora..."
3,38360,1501,East,NaN,Parks,Highway,NaN,Yolo County,99654,1501 East Parks Highway,NaN,Wasilla,Alaska,99654,AK,"1501 EAST PARKS HIGHWAY, WASILLA, AK 99654",38.560528,-121.763156,POINT (-121.7631563224778 38.56052828656745),"1501,East Parks Highway , Wasilla, Yolo Count..."
4,38315,300,NaN,NaN,Front,Street,NaN,San Diego County,99901,300 Front Street,NaN,Ketchikan,Alaska,99901,AK,"300 FRONT STREET, KETCHIKAN, AK 99901",32.629730,-116.965571,POINT (-116.9655713542552 32.6297296364391),"300, Front Street , Ketchikan, San Diego Coun..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,7989,514,South,NaN,Beech,Street,NaN,Natrona County,82601,514 S Beech St,NaN,Casper,Wyoming,82601,WY,"514 S BEECH ST, CASPER, WY 82601",42.845210,-106.320969,POINT (-106.3209689997536 42.8452099996773),"514,South Beech Street , Casper, Natrona Coun..."
542,8028,224,NaN,NaN,West,Road,NaN,Weston County,82701,224 West Rd,NaN,Newcastle,Wyoming,82701,WY,"224 WEST RD, NEWCASTLE, WY 82701",43.853611,-104.222385,POINT (-104.2223850003547 43.85361100006953),"224, West Road , Newcastle, Weston County, Wy..."
543,8058,1101,NaN,NaN,Russell,Avenue,NaN,Washakie County,82401,1101 Russell Ave,NaN,Worland,Wyoming,82401,WY,"1101 RUSSELL AVE, WORLAND, WY 82401",44.022437,-107.954171,POINT (-107.9541709997307 44.02243699983148),"1101, Russell Avenue , Worland, Washakie Coun..."
544,53132,1250,West,NaN,College,Drive,NaN,Laramie County,82007,1250 West College Drive,NaN,Cheyenne,Wyoming,82007,WY,"1250 WEST COLLEGE DRIVE, CHEYENNE, WY 82007",41.103219,-104.816870,POINT (-104.8168701585998 41.10321922590939),"1250,West College Drive , Cheyenne, Laramie C..."


In [67]:
nominatim_sample = pd.concat([dropped_territories, df_sample])
nominatim_sample['zip'] = nominatim_sample.zip.apply(lambda x: str(x).zfill(5))
nominatim_sample = nominatim_sample.drop(columns=['ZipCode'])

nominatim_sample_path = os.path.join(abs_path, 'output', 'nominatim_sample_fixed.csv')

nominatim_sample.to_csv(nominatim_sample_path, index=False)

In [68]:
nominatim_sample.loc[nominatim_sample.zip.str.startswith('0')]

,location_id,AddressNumber,StreetNamePreDirectional,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePostDirectional,county,address_1,address_2,city,state,zip,state_abbr,location_source_value,latitude,longitude,geometry,Nominatim_address
16621,16651,615,NaN,NaN,Main,Avenue,NaN,New York County,615 Main Avenue,NaN,Passaic,New Jersey,07055,NJ,"615 Main Avenue, Passaic, New Jersey 07055",40.720470,-74.000928,POINT (-74.000928 40.72047),"615, Main Avenue , Passaic, New York County, ..."
16623,16653,850,NaN,NaN,Main,Street,NaN,Suffolk County,850 Main Street,NaN,Bridgeport,Connecticut,06604,CT,"850 Main Street, Bridgeport, Connecticut 06604",40.854908,-73.195385,POINT (-73.195385 40.854908),"850, Main Street , Bridgeport, Suffolk County..."
